In [5]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from urllib.parse import urljoin
import re
import time

In [6]:
driver =  webdriver.Chrome(executable_path = 'D:\workspace\chromedriver.exe' )

driver.get('https://ev.or.kr/evmonitor')
#time.sleep(5)

In [7]:
count=0
if count==0:
    tmp_len=0
    create_date=[]

In [8]:
sid = []

In [9]:
for k in range(2,19):
    driver.find_element_by_css_selector('#F_SI_DO1')
    driver.implicitly_wait(5)
    driver.find_element_by_css_selector('#F_SI_DO1 > option:nth-child(%s)'%k).click()
    #time.sleep(3)
    driver.find_element_by_css_selector('#statList')
    driver.implicitly_wait(30)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    a = soup.find_all('div',class_='fw_path')
    for link in a:
        temp = str(link['sid'])
        sid.append(temp)


In [10]:
len(sid)

2493

In [427]:
tmp_len2 = len(sid)-tmp_len
    
if tmp_len != len(sid):
    tmp_len2 = len(sid)-tmp_len
for i in range (0,tmp_len2):
    i = time.ctime()
    create_date.append(i)
    
tmp_len = len(sid)
count+=1

In [428]:
print(len(create_date))

2493


In [494]:
#충전소_딕셔너리
c_station = {'sid':[],'name':[], 'u_time':[], 'agency': [],'do':[], 'city':[], 'gu':[], 'add':[],'phone':[], 'fee':[],'etc':[]}
#충전기_딕셔너리
charger = {'cid':[],'c_name':[], 'typ':[], 'charge':[], 'using':[], 'amount':[], 'sid':[]}

In [495]:
count2=0
if count2==0:
    tmp_len_2=0
    c_create_date=[]

In [496]:
for i in sid:
    url_base = 'https://www.ev.or.kr/portal/monitor/evminfo?sid='
    url = url_base+str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    for tmp_using in soup.find_all(class_='table_01'):
        n_tmp = soup.find_all('h2')
        c_station['name'].append(n_tmp[0].get_text())
        c_station['sid'].append(i)

        t_tmp = soup.find_all('em')
        c_station['u_time'].append(t_tmp[0].get_text())

        info_tmp = soup.find_all(class_='table_02')
        dinfo = info_tmp[0].find_all('td')
        c_station['add'].append(dinfo[0].get_text())
        c_station['agency'].append(dinfo[1].get_text())
        c_station['phone'].append(dinfo[2].get_text())
    
        tmp_fee = dinfo[3].get_text()
        tmp_fee = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_fee)
        c_station['fee'].append(tmp_fee)
        
            
        tmp_etc = dinfo[5].get_text()
        tmp_etc = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_etc)
        c_station['etc'].append(tmp_etc)
        
        
        do_tmp = dinfo[0].get_text().split(' ')
        do_tmp = do_tmp[0]
        gu_tmp = dinfo[0].get_text().split(' ')
        if '특별'or'광역' in do_tmp:
            do_tmp = do_tmp[0]+do_tmp[1]
            gu_tmp = gu_tmp[1]
            c_station['gu'].append(gu_tmp)
            c_station['city'].append(do_tmp+'시')
            c_station['do'].append(do_tmp)
        else:
            city_tmp = dinfo[0].get_text().split(' ')[1]
            if '군' in dinfo[0].get_text().split(' ')[1]:
                gu_tmp = gu_tmp[1]
            else:
                gu_tmp = gu_tmp[2]
            c_station['gu'].append(gu_tmp)
            c_station['city'].append(city_tmp)
            c_station['do'].append(do_tmp)
        
        if len(tmp_using)<=5:
            
            n_tmp = soup.find_all('h2')
            charger['c_name'].append(n_tmp[0].get_text())
            charger['cid'].append(i+'0')
            charger['sid'].append(i)

            ty_tmp = soup.find_all('dd')
            charger['typ'].append(ty_tmp[0].get_text())
            
            tmp_amount = dinfo[4].get_text()
            
            tmp_amount = re.findall('\d+', tmp_amount)
            charger['amount'].append(tmp_amount[0])
            
            for u_tmp in soup.find_all(class_='td3'):
                tmp = re.sub('\n', '', u_tmp.get_text())
                charger['using'].append(tmp)
            
    
            td2 = soup.find_all(class_='td2')
            for te in td2[0:]:
                test=[]
                for a in te.find_all('span'):
                    if a.get_text()=='사용가능':
                        break
                    elif a.get_text()=='사용중':
                        break
                    elif a.get_text()=='사용불가':
                        break
                    elif a.get_text()==' ':
                        break
                    elif len(a.get_text())<7:
                        tmp=[]
                        tmp.append(a.get_text())
                        #print(tmp)
                        test+=tmp
                test= str("+".join(test))
                charger['charge'].append(test)
                if test!=None:
                    break

                
            
        elif len(tmp_using)<=7:
            tmp_using = soup.find_all(class_='ev_type t03')
            t=0
            for te in tmp_using[0:]:
                for a in te.find_all('span'):
                    if a.get_text()==' ':
                        break
                    elif len(a.get_text())<6:
                        charger['charge'].append(a.get_text())

                        n_tmp = soup.find_all('h2')
                        charger['c_name'].append(n_tmp[0].get_text())
                        charger['cid'].append(i+str(t))
                        t+=1
                        charger['sid'].append(i)
                        ty_tmp = soup.find_all('dd')
                        charger['typ'].append(ty_tmp[0].get_text())

                        tmp_amount = dinfo[4].get_text()
            
                        tmp_amount = re.findall('\d+', tmp_amount) #숫자만 추출
                        charger['amount'].append(tmp_amount[0])
                    
                        for u_tmp in soup.find_all(class_='td3'):
                            tmp = re.sub('\n', '', u_tmp.get_text())
                            charger['using'].append(tmp)
                            break


ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다

In [489]:
import time

In [490]:
tmp_len3 = len(charger['c_name'])-tmp_len_2
    
if tmp_len3 != len(charger['c_name']):
    tmp_len3 = len(charger['c_name'])-tmp_len_2
for i in range (0,tmp_len3):
    i = time.ctime()
    c_create_date.append(i)

tmp_len3 = len(charger['c_name'])
count2+=1

In [491]:
len(c_station['name'])

100

In [492]:
c_station['gu']

['화천군',
 '인제군',
 '인제군',
 '속초시',
 '삼척시',
 '삼척시',
 '춘천시',
 '춘천시',
 '속초시',
 '속초시',
 '속초시',
 '삼척시',
 '정선군',
 '홍천군',
 '삼척시',
 '삼척시',
 '횡성군',
 '양구군',
 '양구군',
 '양구군',
 '양구군',
 '양구군',
 '양구군',
 '양양군',
 '양양군',
 '양양군',
 '양양군',
 '양양군',
 '양양군',
 '속초시',
 '춘천시',
 '영월군',
 '영월군',
 '평창군',
 '강릉시',
 '철원군',
 '횡성군',
 '삼척시',
 '삼척시',
 '원주시',
 '원주시',
 '원주시',
 '원주시',
 '인제군',
 '인제군',
 '정선군',
 '강릉시',
 '정선군',
 '고성군',
 '춘천시',
 '삼척시',
 '평창군',
 '춘천시',
 '철원군',
 '철원군',
 '횡성군',
 '춘천시',
 '동해시',
 '춘천시',
 '춘천시',
 '춘천시',
 '춘천시',
 '원주시',
 '원주시',
 '원주시',
 '원주시',
 '원주시',
 '원주시',
 '원주시',
 '태백시',
 '태백시',
 '태백시',
 '원주시',
 '춘천시',
 '홍천군',
 '평창군',
 '평창군',
 '평창군',
 '평창군',
 '평창군',
 '평창군',
 '춘천시',
 '강릉시',
 '춘천시',
 '삼척시',
 '양양군',
 '원주시',
 '인제군',
 '원주시',
 '원주시',
 '춘천시',
 '춘천시',
 '고성군',
 '홍천군',
 '홍천군',
 '홍천군',
 '홍천군',
 '홍천군',
 '홍천군',
 '화천군']

In [493]:
df = pd.DataFrame({'ID': c_station['sid'], '이름':c_station['name'], '사용가능시간':c_station['u_time'], '운영기관':c_station['agency'],'도':c_station['do'],'시':c_station['city'], '구': c_station['gu'],'도로명주소': c_station['add'],'연락처': c_station['phone'],'충전요금': c_station['fee'], '참고사항': c_station['etc']})
df

,ID,이름,사용가능시간,운영기관,도,시,구,도로명주소,연락처,충전요금,참고사항
0,ME18B206,선등거리 주차타워,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,화천군,강원도 화천군 화천읍 하리 16-6,1661-9408,유료,없음
1,ME181409,설악산국립공원 장수대분소,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,인제군,강원도 인제군 북면 한계리 산1-59,1661-9408,유료,없음
2,ME18B208,설악산국립공원점봉산분소,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,인제군,강원도 인제군 인제읍 귀둔리 산242-18,1661-9408,유료,없음
3,ME18B207,설악항 공영주차장,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,속초시,강원도 속초시 동해대로 3664,1661-9408,유료,"최초10분무료, 최초30분-600원 이후10분마다 300원"
4,ME18B107,성내12통 공영주차장,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,삼척시,강원도 삼척시 당저동1길 17-9,1661-9408,유료,없음
...,...,...,...,...,...,...,...,...,...,...,...
95,42720002,홍천군청,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,홍천군,강원도 홍천군 홍천읍 석화로 93,1661-9408,유료,없음
96,ME18B235,홍천생명건강과학관 버스주차장,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,홍천군,강원도 홍천군 홍천읍 연봉리 404-65,1661-9408,유료,없음
97,ME181441,홍천휴게소 서울방향,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,홍천군,강원도 홍천군 화촌면 군업리 220-10(서울방향),1661-9408,유료,없음
98,ME181440,홍천휴게소 양양방향,24시간 이용가능,환경부(한국자동차환경협회),강원,강원시,홍천군,강원도 홍천군 화촌면 군업리 537(양양방향),1661-9408,유료,없음


In [454]:
c_df = pd.DataFrame({'ID':charger['cid'],'이름':charger['c_name'], '구분':charger['typ'], '충전기타입':charger['charge'], '사용중':charger['using'], '급속충전량':charger['amount'], '생성시간':c_create_date, 'sID':charger['sid']})
c_df

,ID,이름,구분,충전기타입,사용중,급속충전량,생성시간,sID
0,ME18B1780,DMC 산학협력연구센터,급속,DC차데모+AC3상+DC콤보,사용가능,100,Wed Sep 16 15:54:26 2020,ME18B178
1,ME19A2460,Park M (주차복합빌딩),급속,DC콤보,사용중,200,Wed Sep 16 15:54:26 2020,ME19A246
2,ME19A2461,Park M (주차복합빌딩),급속,DC콤보,사용중,200,Wed Sep 16 15:54:26 2020,ME19A246
3,ME18B1700,SETEC,급속,DC차데모+AC3상+DC콤보,사용가능,100,Wed Sep 16 15:54:26 2020,ME18B170
4,ME18B2780,sh서울주택도시공사,급속,DC콤보,사용중,200,Wed Sep 16 15:54:26 2020,ME18B278
...,...,...,...,...,...,...,...,...
2652,ME19B1710,홍로공영주차장,급속,DC콤보,사용중,100,Wed Sep 16 15:54:26 2020,ME19B171
2653,ME1740820,화북동주민센터,급속,DC차데모+AC3상+DC콤보,사용중,50,Wed Sep 16 15:54:26 2020,ME174082
2654,501101210,환경성질환예방관리센터,급속,DC차데모+AC3상+DC콤보,사용가능,50,Wed Sep 16 15:54:26 2020,50110121
2655,ME1740620,회수사거리 인근 공영주차장,급속,DC차데모+AC3상+DC콤보,사용가능,50,Wed Sep 16 15:54:26 2020,ME174062


In [455]:
df.to_csv('station.csv',encoding='euc-kr')
c_df.to_csv('charger.csv',encoding='euc-kr')

In [456]:
import pymysql
# MySQL Connector using pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

In [457]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='1234',
                       db='test', charset='utf8')
cur = conn.cursor()

In [458]:
a=0

In [459]:
tmp_station={}
tmp_charger={}

In [460]:
if a==0:
    tmp_station={'name':[]}
    tmp_charger={'c_name':[]}
    cd=[]

In [461]:
#충전소
#딕셔너리 for문
for i in range(0,len(c_station['name'])):
    for j in range(0, len(tmp_station['name'])):
        if len(tmp_station) != 1:
            for t in tmp_station.keys():
                if tmp_station[t][j]!=c_station[t][j]:
                    sql="update charge_station set "+t+"='%s' where "+t+"='%s'"%(t[j], tmp_station[t][j])
                    cur.execute(sql)
            if cd[j]!=create_date[j]:
                sql="update charging_station set create_date='%s' where name='%s'"%(create_date[j], tmp_station['name'][j])
                cur.execute(sql)
    if len(tmp_station['name'])!=len(c_station['name']):
        sql="""insert into charging_station select '%s','%s', '%s', '%s','%s','%s','%s','%s','%s','%s','%s','%s' from dual where not exists (select * from charging_station where sid='%s')""" % (c_station['sid'][i], c_station['name'][i], c_station['u_time'][i], c_station['agency'][i],c_station['do'][i], c_station['city'][i], c_station['gu'][i], c_station['add'][i], c_station['phone'][i], c_station['fee'][i], c_station['etc'][i], create_date[i], c_station['sid'][i])
        cur.execute(sql)

In [464]:
#충전기
for i in range(0,len(charger['c_name'])):
    tmp_use = {'id':charger['cid'], 'name':charger['c_name'], 'c_using':charger['using']}
    for j in range(0, len(tmp_charger['c_name'])):
        if len(tmp_charger) != 1:
            for t in tmp_charger.keys():
                if tmp_charger[t][j]!=charger[t][j]:
                    sql="update charger set "+t+"='%s' where "+t+"='%s'"%(t[j], tmp_charger[t][j])
                    cur.execute(sql)
            #if cd[j]!=create_date[j]:
            #    sql="update charging_station set create_date='%s' where name='%s'"%(create_date[j], dic['name'][j])
            #    cur.execute(sql)
    if len(tmp_charger['c_name'])!=len(charger['c_name']):
        sql="""insert into charger (id, sid, c_name, typ, charge, c_using, amount, c_create_date) select '%s','%s','%s', '%s', '%s','%s','%s','%s' from dual where not exists (select * from charger where id='%s')""" % (charger['cid'][i],charger['sid'][i],charger['c_name'][i], charger['typ'][i], charger['charge'][i],charger['using'][i],charger['amount'][i], c_create_date[i], charger['cid'][i])
        cur.execute(sql)

In [465]:
conn.commit()

In [466]:
a=1
tmp_station = c_station
tmp_charger = charger
cd= create_date

In [467]:
import time

In [468]:
status=[]
first=0

In [469]:
for i in range(0,len(tmp_use['name'])):
    if first==0:
        if charger['using'][i]=='사용중':
            status.append(1)
            date=time.strftime('%Y-%m-%d ')+time.ctime().split(' ')[0]
            u_time=time.strftime("%H:%M:%S")
            sql="""insert into usage_status (id, start_date, start_time) values ('%s', '%s', '%s') """ % (tmp_use['id'][i], date ,u_time)
            cur.execute(sql)
        else:
            status.append(0)
    else:
        if charger['using'][i]=='사용중' and status[i]==0:
            date=time.strftime('%Y-%m-%d ')+time.ctime().split(' ')[0]
            u_time=time.strftime("%H:%M:%S")
            sql="""insert into usage_status (id, start_date, start_time) values ('%s', '%s', '%s') """ % (tmp_use['id'][i], date ,u_time)
            cur.execute(sql)
        if charger['using'][i]=='사용가능':
            status[i]=0
            date=time.strftime('%Y-%m-%d ')+time.ctime().split(' ')[0]
            p_time=time.strftime("%H:%M:%S")
            sql="update usage_status set end_date='%s', end_time='%s', use_time='%s' where id='%s' and end_date IS NULL"%(date, p_time, '3', tmp_use['id'][i])
            cur.execute(sql)
first=1

In [470]:
conn.commit()

In [471]:
conn.close()